<a href="https://colab.research.google.com/github/MuraliKrishnadata/google_colab_working_files/blob/main/Arima_Assinment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple ARIMA model for non-seasonal time-serie forecast

Our goal in this challenge is to apply the basic concepts of time series analysis on one-dimension data

In this challenge, we'll go through the following steps :
1. load and visualize the data;
2. train our models and make predictions;

##  1. Load Data
Let's start by loading the time series of this real use case: Daily sales of a restaurant

In [1]:
!unzip "/content/drive/MyDrive/Machine_Learing_files/AI_ML_for_Time_series_forcasting/Re_ [IMPORTANT] Séances Times Series Forecasting Promo DSBA EDC Mars 2025.zip"

unzip:  cannot find or open /content/drive/MyDrive/Machine_Learing_files/AI_ML_for_Time_series_forcasting/Re_ [IMPORTANT] Séances Times Series Forecasting Promo DSBA EDC Mars 2025.zip, /content/drive/MyDrive/Machine_Learing_files/AI_ML_for_Time_series_forcasting/Re_ [IMPORTANT] Séances Times Series Forecasting Promo DSBA EDC Mars 2025.zip.zip or /content/drive/MyDrive/Machine_Learing_files/AI_ML_for_Time_series_forcasting/Re_ [IMPORTANT] Séances Times Series Forecasting Promo DSBA EDC Mars 2025.zip.ZIP.

No zipfiles found.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

# Reading the data from souce to pandas dataframe
resturant_data = pd.read_csv('/content/Daily_restaurant_sales.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/Daily_restaurant_sales.csv'

In [ ]:
resturant_data.sample(10)

In [ ]:
resturant_data.shape

This abstract time serie does not seem seasonal, but with some increasing trend and somehow "sticky" (i.e. with some auto-regressivity). So it may be a good candidate for Auto-Regresive Moving Average (ARIMA) models


## 2. Build ARIMA model
We will try to forecast the data thanks to ARIMA models (Auto Regressive Integrated Moving Average).

For that, we will need to :
1. find how to stationarize the time serie (I in SARIMA)
2. find the auto-regressive (AR) part
3. find the Moving-Average (MA) part
4. Fit
5. Assess performance

### Step 1 - Ensure stationarity

ARIMA models applies to "stationary" time series only.

👉 Check its stationarity precisely using the [`Augmented Dick Fuller test`](https://www.statsmodels.org/stable/generated/statsmodels.tsa.stattools.adfuller.html), and especially its p-value

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Perform the Augmented Dickey-Fuller test
result = adfuller(resturant_data['x'])

# Print the results
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

The p-value should be  less than 0.05 to have a 95% confidence in the stationarity.  
If the p-value is larger than 0.05, we cannot reject the null hypothesis (null hypothesis = "the process is not stationary").

If the time series is not stationary, it needs to be stationarized through **differencing**.
- It means that we take the difference between each value and the preceding one (*first difference*).
- Repeat process on the differentiated serie if you want the *second difference*, etc...

👉 Find the minimum order of differencing we need to make it stationnary (plot the curves to visualize them, and print their adfuller p-value to be sure)

<details>
    <summary>Hint</summary>

`pd.Series.diff`
</details>

In [ ]:
import matplotlib.pyplot as plt

# Calculate the first difference
resturant_data_diff_1 = resturant_data['x'].diff().dropna()

# Calculate the second difference
resturant_data_diff_2 = resturant_data_diff_1.diff().dropna()

# Plot the original and differenced series
plt.figure(figsize=(12, 6))
plt.plot(resturant_data['x'], label='Original')
plt.plot(resturant_data_diff_1, label='First Difference')
plt.plot(resturant_data_diff_2, label='Second Difference')
plt.legend()
plt.title('Original and Differenced Time Series')
plt.show()

# Perform ADF test on the first difference
result_diff_1 = adfuller(resturant_data_diff_1)
print('ADF Statistic (First Difference): %f' % result_diff_1[0])
print('p-value (First Difference): %f' % result_diff_1[1])

# Perform ADF test on the second difference
result_diff_2 = adfuller(resturant_data_diff_2)
print('ADF Statistic (Second Difference): %f' % result_diff_2[0])
print('p-value (Second Difference): %f' % result_diff_2[1])

We have a close-call here between one and two diff orders. Differentiating too much a time series may also reduce performance of your ARIMA models. Let's have a closer look:

👉 Plot autocorrelation plot ([`plot_acf`](https://www.statsmodels.org/stable/generated/statsmodels.graphics.tsaplots.plot_acf.html)) for diff order 1 and 2.

(💡Pro tip: Avoid duplicating statsmodels plots by calling `plt.show()` or by adding `;` to the end of each instantiation of a statsmodels plot)

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

#First Auto correlation
resturant_data_diff_1 = resturant_data['x'].diff().dropna()
plt.figure(figsize=(12, 20))
plot_acf(resturant_data_diff_1, lags=40)
plt.title('Autocorrelation Plot (First Difference)')
plt.show();

In our "second order diff" autocorrelation plot, the lag coefficient n°1 is close to 0, while the second one escapes far into negative territory. This might indicate we have over-differentiated the series. (Remember: we never care about the lag n°0 which is always equal to 1)

👉 Let's (temptatively) keep only one diff order and name this series `y_diff` (we can always try more diff later)

In [ ]:
#Second order defference
resturant_data_diff_2 = resturant_data_diff_1.diff().dropna()
plt.figure(figsize=(12, 6))
plot_acf(resturant_data_diff_2, lags=40)
plt.title('Autocorrelation Plot (Second Difference)') # Changed title for clarity
plt.show();

We just found the term "I" in ARIMA: `d = 1` for 1-diff before stationary (I refers to "integration", "d" for differentiation...)


### Step 2 - Select AR order (p) and MA order (q)

#### MA($\color{blue}{q}$) = number of lag beyond which the $\color{blue}{ACF}$ of  $Y^{\color{green}{(d)}}$ cuts off

MA order (`q`) can be found by looking at the autocorrelation plot ([`plot_acf`](https://www.statsmodels.org/stable/generated/statsmodels.graphics.tsaplots.plot_acf.html)) applied to`y_diff`.

👉 determine `q`

In [ ]:


# Assuming y_diff is the first difference of the data
# If y_diff is not defined, use resturant_data['x'].diff().dropna()
y_diff = resturant_data['x'].diff().dropna()


# Plot the ACF of y_diff
plot_acf(y_diff, lags=20)
plt.title('Autocorrelation Plot (y_diff)')
plt.show()

We could choose q = 4, but it's more conservatively to try with `q=2` to start with.

When in doubt, go with the simpler model that sufficiently explains the Y.

#### AR($\color{red}{p}$) = number of lags beyond which the $\color{red}{PACF}$ of $Y^{\color{green}{(d)}}$  cuts off

AR order (`p`) can be found by investigating the **p**artial autocorrelation plot [`plot_pacf`](https://www.statsmodels.org/stable/generated/statsmodels.graphics.tsaplots.plot_pacf.html) applied to `y_diff`.

(Partial autocorrelation can be imagined as the correlation between the series and its lag, after excluding the contributions from the intermediate lags. So, PACF sort of conveys the pure correlation between a lag and the series)

👉 Determine `p`

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
import matplotlib.pyplot as plt

# Assuming y_diff is the first difference of the data
# If y_diff is not defined, use resturant_data['x'].diff().dropna()
y_diff = resturant_data['x'].diff().dropna()

# Plot the PACF of y_diff
plot_pacf(y_diff, lags=20)
plt.title('Partial Autocorrelation Plot (y_diff)')
plt.show()

We could choose `p = 3` as the first 3 lag terms seems above the significance level, but we could also go with a simpler model `p = 1`.



### Step 3 - Build the model

Now that you have chosen the values for `p`, ` d` and `q` for the ARIMA,

👉 build the `arima_model` from `statsmodels`.
- fit the the model
- print the model (`.summary`)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
arm_model=ARIMA(resturant_data['x'],order=(1,1,2))
model_fit=arm_model.fit()
print(model_fit.summary())

☝️ If your p-values are too high, try remove these terms by reducing the corresponding AR or SA coefficients.

You can evaluate overall performance of your fit by minimizing the [`AIC - akaike information criterion`](https://towardsdatascience.com/the-akaike-information-criterion-c20c8fd832f2) value

It seems that the (1,1,1) ARIMA models have less chance of overfitting (p-values remains low) and maintain a quasi similar AIC score than other models

## 3. Evaluate model performance

👉 Visualize your model predictions with `.plot_predict()`

- Look closely at the method default params, especially `dynamic`.
- Do you think your model would have such good performance in reality?

In [ ]:
# Make predictions using the fitted model
predictions = model_fit.predict(start=0, end=len(resturant_data['x']) - 1, dynamic=False)

# Plot the original data and the predictions
plt.figure(figsize=(12, 6))
plt.plot(resturant_data['x'], label='Original')
plt.plot(predictions, label='Predictions (dynamic=False)')
plt.title('ARIMA Model Predictions (dynamic=False)')
plt.legend()
plt.show()

☝️ `dynamic=False` actually makes use of all available values `y` in order to predict `y_pred`, making your ARIMA prediction use up to $y_{t-1}$ to predict $y_t$. In reality, you don't have access to all `y`, especially if you want to predict several intervals in the future.

👉 Try to use `dynamic=True` to plot a prection of the _last 15 values_ in a situation where the model only have _access to data up to 85_. That is to say, the model:
- predicts 86 based on true [1...85]
- then predicts 87 based on [1...85] _plus_ it's previouly predicted value for 86
- etc...iteratively until 100

In [ ]:
# Make predictions with dynamic=True for the last 15 values
# The model has access to data up to index 85
predictions_dynamic = model_fit.predict(start=85, end=len(resturant_data['x']) - 1, dynamic=True)

# Plot the original data and the dynamic predictions
plt.figure(figsize=(12, 6))
plt.plot(resturant_data['x'], label='Original')
plt.plot(predictions_dynamic, label='Predictions (dynamic=True)')
plt.title('ARIMA Model Predictions (dynamic=True)')
plt.legend()
plt.show()

☝️ That's still not a _true_ forecast!! Why??

<details>
    <summary>Answer</summary>

Our model has "seen" the whole `y_true` serie during the fitting phase!
</details>

### 3.1 Out-of-sample forecasts (real "future")

👉 Create a train-test-split keep the last 15 datapoints only for the test set, and train your ARIMA on the train set only.

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
train, test = train_test_split(resturant_data, test_size=15, shuffle=False)

👉 We are "now" in step 85 and have never seen the future:
- Use `.forecast()` method on your fitted model to "forecast" the 15 next datapoints (i.e beyond the end of your train dataset)
- Plot forecasted values as well as the higher and lower range of 95% uncertainty interval

In [ ]:
forcast=model_fit.forecast(steps=15)

In [ ]:
# Get the forecast with confidence intervals
forecast_result = model_fit.get_forecast(steps=15)
forecast_mean = forecast_result.predicted_mean
conf_int = forecast_result.conf_int(alpha=0.05) # 95% confidence interval

plt.figure(figsize=(12, 6))
plt.plot(resturant_data['x'][:85], label='Original (Train)') # Plot training data
plt.plot(resturant_data['x'][85:], label='Original (Test)') # Plot test data
plt.plot(forecast_mean, label='Forecasted')
plt.fill_between(conf_int.index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='k', alpha=0.1, label='95% Confidence Interval')

plt.title('ARIMA Model Forecast with 95% Confidence Interval')
plt.legend()
plt.show()

👉 Try to also plot your previous 85 `y` real datapoints to better grasp model performance relative to the past

In [ ]:
# Define y_real and y_pred for plotting
y_real = resturant_data['x'][85:] # Test data
y_pred = forecast_mean # Forecasted values

plt.figure(figsize=(12, 6)) # Increased figure size for better readability
plt.plot(range(len(resturant_data)), resturant_data['x'], label='Actual', color='blue')
plt.plot(range(85, 85 + len(y_pred)), y_pred, label='Predicted', color='red', linestyle='--') # Adjusted x-axis for predicted values
plt.title("Model Performance vs Historical Data")
plt.xlabel("Time")
plt.ylabel("Values")
plt.legend()
plt.grid(True)
plt.show()

### 3.2 Can you trust your 95% confidence interval? (conditions for inference)

👉 Plot the residuals `model.resid` to ensure there are no patterns
- Normally distributed
- Mean zero
- Uniform variance
- No autoregressive patterns (you can plot_acf the residuals if you really want)

Note: residuals are constructed by 'seing' all data as in `plot_predict(dynamic=False)`

Also try to plot a histogram or kde fit of the residuals to see if they are approximately normally distributed.

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(model_fit.resid, label='Residuals')
plt.title('Residuals Plot')
plt.legend()
plt.show()

## 3.3 Cross-validated performance metrics

👉 Below are the given the most common performance metrics for time series

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score,mean_squared_error


In [ ]:
# Calculate performance metrics
mse = mean_squared_error(y_real, y_pred)
mae = mean_absolute_error(y_real, y_pred)
r2 = r2_score(y_real, y_pred)

print(f'Mean Squared Error (MSE): {mse:.2f}')
print(f'Mean Absolute Error (MAE): {mae:.2f}')
print(f'R-squared (R2): {r2:.2f}')

👉 Play with you ARIMA hyper-parameters and see the impact on your forecast performance

## 4 Grid Search
#link to understand Grid Search https://www.lovelyanalytics.com/2017/10/16/grid-search/

Try to run your own gridsearch for (p,d,q) using `pmdarima`. Use at least
- `trace=True`
- `error_action='ignore'`
- `suppress_warnings=True`

## Cross-validate performance of your model?

In practice, Results and GridSearch should always be cross validated:

Feel free to use [`sklearn.TimeSeriesSplit`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html) to create continguous K-folds so as to truely evaluate the performance of your model and find the best hyper-params after cross validation

<img src='https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_0101.png'>